## OSRM (Open Source Routing Machine)

## Table of Contents

- [Trip Service](#trip-service)
- [Loading the Data](#loading-the-data)
- [Plotting the Data](#plotting-the-data)
- [Using the Service](#using-the-service)
  - [General Options](#general-options)
  - [Additional Options](#additional-options)
  - [Making the Request](#making-the-request)
  - [No Source & No Destination](#no-source--no-destination)
  - [Fixed Source & Fixed Destination](#fixed-source--fixed-destination)
- [Conclusion](#conclusion)

## Trip Service

The **Trip** service is used to solve the Traveling Salesman Problem (TSP).  
It computes an optimized route that visits all provided waypoints in the most efficient order.

This is useful for:
- Delivery route optimization  
- Multi-stop logistics planning  
- Field service scheduling  
- Any scenario requiring optimal waypoint ordering  

You can read more about the service in the official OSRM documentation.

Since I am working with the Bremen map, I selected several locations within the city to demonstrate how the Trip service works. If you are working with a different region, make sure to use coordinates from your own area.

In [ ]:
import pandas as pd
from pydantic import BaseModel


class Point(BaseModel):
    latitude: float
    longitude: float


df = pd.read_csv("../data/trip_service_data.csv")
points = []
for i, row in df.iterrows():
    latitude = row["latitude"]
    longitude = row["longitude"]
    points.append(Point(latitude=latitude, longitude=longitude))

points[:5]

## Loading the Data

For this demonstration, we use a dataset of cities to showcase how the Trip service works.

In this example, the data was selected from the Morocco map to simulate a multi-city routing scenario.  
If you are working with a different region, make sure to use coordinates from your own dataset.

The CSV file contains latitude and longitude columns, which are converted into structured `Point` objects before sending them to OSRM.

Also make sure to add the coorrdinates of your selected region inisde the *"trip_service_data.csv"* file

In [ ]:
import pandas as pd
from pydantic import BaseModel


class Point(BaseModel):
    latitude: float
    longitude: float


df = pd.read_csv("../data/trip_service_data.csv")
points = []
for i, row in df.iterrows():
    latitude = row["latitude"]
    longitude = row["longitude"]
    points.append(Point(latitude=latitude, longitude=longitude))

points[:5]

### Plotting the Data

To better understand the selected locations, we visualize them on an interactive map using the `folium` library.

The map is centered on the first point in the dataset, and each location is marked with a blue marker. This allows us to visually inspect the spatial distribution of the selected cities in Bremen before applying the Trip service.

Visualizing the raw points helps ensure that:
- The coordinates are correct  
- The points are located within the expected region  
- The dataset is suitable for route optimization  

This map serves as the foundation for the next step, where we compute the optimal trip connecting these locations.

In [ ]:
import folium

folium_map = folium.Map(
    location=[points[0].latitude, points[0].longitude], zoom_start=6)

for point in points:
    folium.Marker(
        location=[point.latitude, point.longitude],
        icon=folium.Icon(color='blue')
    ).add_to(folium_map)

folium_map

## Using the Service

### General Options

The general options are common across all OSRM services.  
The table below summarizes the core parameters used when constructing a Trip request.

| Parameter    | Description |
|-------------|-------------|
| `service`   | One of the following values: `route`, `nearest`, `table`, `match`, `trip`, `tile` |
| `version`   | Version of the protocol implemented by the service. Typically `v1` for OSRM 5.x installations. |
| `profile`   | Mode of transportation, determined by the Lua profile used during data preparation (e.g., `car`, `bike`, `foot`). |
| `coordinates` | String formatted as `{longitude},{latitude};{longitude},{latitude};...` or `polyline({polyline})`. |
| `format`    | Only `json` is supported. This parameter is optional and defaults to `json`. |

### Additional Options

The Trip service provides additional parameters that control how the Traveling Salesman Problem (TSP) is solved:

- `roundtrip`  
  A boolean value indicating whether the returned route should return to the starting point.

- `source`  
  Specifies the starting location of the trip.  
  Can be either `any` (default) or `first`.

- `destination`  
  Specifies the ending location of the trip.  
  Can be either `any` (default) or `last`.

### Making the Request

The Trip endpoint follows this structure:

### No Source & No Destination

This is the default behavior.  
If neither `source` nor `destination` is specified:

- OSRM automatically determines the optimal starting and ending points.
- If `roundtrip=true` (default), the trip will start and end at the same location.
- If `roundtrip=false`, OSRM will choose the best start and end points to minimize total travel cost.

This configuration allows OSRM to fully optimize the route without constraints.

In [ ]:
service = 'trip'
version = 'v1'
profile = 'driving'
host = 'http://localhost:5000'

In [ ]:
import requests

coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])
url = f'{host}/{service}/{version}/{profile}/{coordinates}'
params = {
    "steps": "true",
    "geometries": "geojson",
    "overview": "full",
    "annotations": "true",
}

response = requests.get(url, params=params)
response.status_code

After sending the request, we receive a successful response (`200`).  
The optimized route can be extracted from the `trips` field in the JSON response.  
The Trip service computes the fastest possible tour that visits all provided coordinates.

In [ ]:
data = response.json()
data.keys()

We inspect the response structure to understand what was returned:

- `code` → Status of the request  
- `trips` → The optimized trip(s)  
- `waypoints` → The reordered waypoints used in the optimized solution  

Since only one trip is requested, we extract the first element from the `trips` list.

In [ ]:
trip = data['trips'][0]
trip.keys()

The extracted `trip` object contains detailed routing information, including:

- `legs` → Individual segments between consecutive waypoints  
- `geometry` → Encoded or GeoJSON route geometry  
- `distance` → Total travel distance  
- `duration` → Total travel time  
- `weight` → Optimization cost  
- `weight_name` → Cost metric used for optimization  

This structure is similar to the response returned by the Route service in the first notebook.

Next, we define helper functions to improve map visualization.

These functions will:

- Add styled markers to represent waypoints  
- Draw route polylines  
- Create custom HTML popups for better readability  

This allows us to clearly visualize the optimized trip on the interactive map.

In [ ]:
from folium import Map


def create_marker(
    map_obj: Map,
    point: Point,
    label: str,
    background_color: str,
    y_offset: int = 0
) -> None:
    folium.Marker(
        location=[point.latitude, point.longitude],
        icon=folium.DivIcon(html=f'''
            <div style="
                background-color: {background_color};
                border-radius: 5px;
                padding: 2px 5px;
                font-size: 8pt;
                text-align: center;
                width: 20px;
                height: 20px;
                line-height: 20px;
                transform: translate(0px, {y_offset}px);
                ">
                {label}
            </div>
        ''')
    ).add_to(map_obj)


def add_polyline(
    map_obj: Map,
    coordinates: list[Point],
    color: str,
    popup_html: str
) -> None:
    folium.PolyLine(
        locations=[[point.latitude, point.longitude] for point in coordinates],
        color=color,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(map_obj)


def get_popup_html(leg: dict, i: int) -> str:
    distance_km = leg['distance'] / 1000
    duration_sec = leg['duration']
    hours, remainder = divmod(duration_sec, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"""Leg number: {i + 1}<br>Distance: {distance_km:.2f} km<br>Duration: {int(hours):02}:{int(minutes):02}:{int(seconds):02}"""

The starting point of the trip is highlighted in **green**, while the ending point is highlighted in **red**.

In this example, both markers appear at the same location. This happens because, by default, the Trip service runs with `roundtrip=true`. As a result, OSRM generates a closed tour — the route begins at a point and returns to that same point at the end of the trip.

This behavior ensures that all waypoints are visited while minimizing the total travel cost in a complete loop.

In [ ]:
folium_map = folium.Map(
    location=[points[0].latitude, points[0].longitude], zoom_start=6)

colors = ["red", "black"]
legs = trip['legs']

for i, leg in enumerate(legs):
    start_coords = leg["steps"][0]["geometry"]["coordinates"][0]
    starting_point = Point(latitude=start_coords[1], longitude=start_coords[0])
    create_marker(
        map_obj=folium_map,
        point=starting_point,
        label=i+1,
        background_color="#93ff6a" if i == 0 else "#6aaeff",
        y_offset=10 if i == 0 else 0
    )

    for step in leg["steps"]:
        coordinates = [Point(latitude=lat, longitude=lon, timestamp=0)
                       for lon, lat in step["geometry"]["coordinates"]]

        add_polyline(
            map_obj=folium_map,
            coordinates=coordinates,
            color=colors[i % len(colors)],
            popup_html=get_popup_html(leg=leg, i=i)
        )

end_coords = legs[-1]["steps"][-1]["geometry"]["coordinates"][-1]
end_point = Point(latitude=end_coords[1], longitude=end_coords[0])
create_marker(
    map_obj=folium_map,
    point=end_point,
    label=len(legs) + 1,
    background_color="#ff6a6a",
    y_offset=-10
)

folium_map

### Fixed Source & Fixed Destination

In this configuration, we explicitly control the start and end points of the trip.

To achieve this, we add three additional parameters to the request:

- `source="first"` → forces the trip to start at the first coordinate  
- `destination="last"` → forces the trip to end at the last coordinate  
- `roundtrip="false"` → prevents the route from returning to the starting point  

By disabling the roundtrip behavior and fixing both endpoints, OSRM computes the shortest path that begins at the first location and terminates at the last location.

As observed in the visualization, the starting and ending markers are now different, confirming that the trip is no longer a closed loop.

In [ ]:
coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])
url = f'{host}/{service}/{version}/{profile}/{coordinates}'
params = {
    "steps": "true",
    "geometries": "geojson",
    "overview": "full",
    "annotations": "true",
    "source": "first",
    "destination": "last",
    "roundtrip": "false",
}

response = requests.get(url, params=params)
data = response.json()
trip = data['trips'][0]

As shown in the visualization, the starting and ending points are no longer identical.

This confirms that the route is not a roundtrip and that OSRM respected the fixed `source` and `destination` parameters defined in the request.

In [ ]:
folium_map = folium.Map(
    location=[points[0].latitude, points[0].longitude], zoom_start=6)

colors = ["red", "black"]
legs = trip['legs']

for i, leg in enumerate(legs):
    start_coords = leg["steps"][0]["geometry"]["coordinates"][0]
    starting_point = Point(latitude=start_coords[1], longitude=start_coords[0])
    create_marker(
        map_obj=folium_map,
        point=starting_point,
        label=i+1,
        background_color="#93ff6a" if i == 0 else "#6aaeff",
        y_offset=0
    )

    for step in leg["steps"]:
        coordinates = [Point(latitude=lat, longitude=lon, timestamp=0)
                       for lon, lat in step["geometry"]["coordinates"]]

        add_polyline(
            map_obj=folium_map,
            coordinates=coordinates,
            color=colors[i % len(colors)],
            popup_html=get_popup_html(leg=leg, i=i)
        )

end_coords = legs[-1]["steps"][-1]["geometry"]["coordinates"][-1]
end_point = Point(latitude=end_coords[1], longitude=end_coords[0])
create_marker(
    map_obj=folium_map,
    point=end_point,
    label=len(legs) + 1,
    background_color="#ff6a6a",
    y_offset=0
)

folium_map

## Conclusion

In this notebook, we explored the OSRM **Trip** service
and demonstrated how it can be used to solve
Traveling Salesman Problem (TSP)-like scenarios.

We covered:

- Loading city coordinate data from a CSV file  
- Visualizing raw locations on an interactive map  
- Understanding the general and additional Trip service parameters  
- Executing a default roundtrip request  
- Extracting trip legs, geometry, and route metadata  
- Visualizing the optimized trip with colored legs  
- Fixing the source and destination points  
- Disabling roundtrip behavior for directional routing  

We observed how OSRM automatically optimizes
the visiting order of waypoints to minimize total travel cost.

By experimenting with parameters such as
`roundtrip`, `source`, and `destination`,
we gained full control over route behavior —
whether returning to the origin or defining
explicit start and end points.

The Trip service is particularly powerful for
logistics planning, delivery optimization,
and multi-stop routing problems where
finding the optimal visit order is essential.